#

<div align=center>
<img src="https://uol.unifor.br/acesso/app/autenticacao/assets/img/logos/icon-unifor.svg" width=45 height=45>
<img src="https://vortex.unifor.br/assets/logos/v1.png" width=45 height=45>
<img src="https://ouch-cdn2.icons8.com/YGkdXgv0NLHfLDlV7bbprOdJuob-J4_zBkql2tweSAk/rs:fit:296:456/extend:false/wm:1:re:0:0:0.8/wmid:ouch/czM6Ly9pY29uczgu/b3VjaC1wcm9kLmFz/c2V0cy9zdmcvNjY5/Lzc2NWQ4ZWU1LWE0/NzItNDMzZi04MGI0/LWZiYWRlZTc1NmVh/Yy5zdmc.png
" width=45 height=45>

<font size=5 color='white'>
<br><br>
<font size=5 color='white'><strong>Projeto:</strong> Pellis

<strong>Etapa:</strong> Treinamento e teste

<strong>Autoria:</strong> Heitor Teixeira

</div>

## 
<font size=5 color='white'> 0 - Bibliotecas e configuração de GPU

In [1]:
import tensorflow as tf
import time
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import GPUtil
from tensorflow.keras.optimizers import Adam, SGD, RMSprop # type: ignore
from tensorflow.keras.losses import CategoricalCrossentropy
from auxiliar.dataset import criar_dataset
from auxiliar.avaliar import avaliar_modelo, matriz_confusao, proximo_id
from auxiliar.callbacks import callbacks
from auxiliar.arquiteturas import custom_model

tf_gpu = tf.config.experimental.list_physical_devices('GPU')
if tf_gpu:
    try:
        nome_gpu = GPUtil.getGPUs()
        print("GPU configurada: ", nome_gpu[0].name)
        tf.config.experimental.set_memory_growth(tf_gpu[0], True)
    except RuntimeError as e:
        print(e)


GPU configurada:  NVIDIA GeForce RTX 4080
Physical devices cannot be modified after being initialized


## 
<font size=5 color='white'> 1 - Hiperparametros

In [2]:
'''
ao lado o batchsize que comporta na minha placa de vídeo, adequa com o que couber.
'''

redes = [
        ['EfficientNetV2B3', 50],  # 0
        ['EfficientNetV2S', 16],   # 1
        ['EfficientNetV2M', 10],   # 2
        ['EfficientNetV2L', 5],    # 3
        ['InceptionV3', 80],       # 4
        ['InceptionResNetV2', 50], # 5
        ['ResNet50V2', 150],       # 6
        ['ResNet152V2', 50],       # 7 
        ['ResNeXt50', 30],         # 8
        ['ResNeXt101', 16]         # 9 
]



In [3]:
seed = 51 # boa ideia 
id_rodada = proximo_id('./rodadas.csv')
n_rede = 3
rede = redes[n_rede][0]
batch_size = redes[n_rede][1]

epocas = 100
loss = CategoricalCrossentropy()
nome_loss = 'CategoricalCrossentropy' # uma string pra salvar no csv
lr = 0.00001

otimizador = Adam(learning_rate = lr) # Adam, SGD, RMSprop
nome_otimizador = 'adam' # uma string pra salvar no csv 

attention = '-' # 'se' ou 'cbam'. as 2 camadas de attention que adicionei caso queria treinar com
camada_pooling = 'global_avg' # 'global_max', 'global_avg'
flatten = False 
imagenet = 'imagenet' # imagenet, imagenet-21k, imagenet-21k-ft1k, os modelos efficient net podem ser treinados na 21k

'''
proejetei pra aceitar ate 5 camadas densas, é so ajustar que da pra aumentar. so acho que nao precisa
'''

denses = [1024, 256] 
dropouts = [0.5, 0.1] 
data_aug = True 


## 
<font size=5 color='white'> 2 - Importando dados e criando datasets

In [4]:
dir_dados_treino = '../pellis_treino.csv'
dir_imagens_treino = '../imagens/treino_imagens'

dir_dados_teste = '../pellis_teste.csv'
dir_imagens_teste = '../imagens/teste_imagens'

dir_modelos_salvos = f'./redes/{rede}/modelos_salvos/{rede}-{id_rodada}_epoca_{{epoch:02d}}.hdf5'
dir_csv_log_treino = f'./redes/{rede}/log_treino/{rede}-{id_rodada}_treinamento_log.csv'

val_size = 0.15

dados_treino = pd.read_csv(dir_dados_treino)
dados_teste = pd.read_csv(dir_dados_teste)

dados_treino, dados_validacao = train_test_split(
    dados_treino,
    random_state=seed,
    test_size = val_size,
)

print(len(dados_treino))
print(len(dados_validacao))
print(len(dados_teste))

1698
300
503


In [5]:
dataset_treino = criar_dataset(
    dataframe = dados_treino,
    diretorio = dir_imagens_treino,
    batch_size = batch_size,
    rede = rede,
    shuffle = True,
    data_aug = data_aug
)

dataset_validacao = criar_dataset(
    dataframe = dados_validacao,
    diretorio = dir_imagens_treino,
    batch_size = batch_size,
    rede = rede,
)

dataset_teste = criar_dataset(
    dataframe = dados_teste,
    diretorio = dir_imagens_teste,
    batch_size = batch_size,
    rede = rede,

)

## 
<font size=5 color='white'> 3 - Modelando a rede

In [6]:
# ja foi comentado nos arquivos auxiliares
model = custom_model(
    rede= rede,
    loss = loss,
    weights = imagenet,
    otimizador = otimizador, 
    attention = attention, 
    denses = denses,
    dropouts = dropouts,
    pooling = camada_pooling,
    flatten=flatten
)

num_layers = len(model.layers)
print(f'camadas no modelo: {num_layers}')

model.summary()

camadas no modelo: 1033
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 stem_conv (Conv2D)             (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 stem_bn (BatchNormalization)   (None, None, None,   128         ['stem_conv[0][0]']              
                                32)                                   

In [7]:
# ja foi comentado nos arquivos auxiliares
early_stop_epocas = 10
check_epocas = 10
reduce_lr_epocas = 2
fator_reduce_lr = 0.7 # fator de reducao, vai multiplicar a lr por ele, se eu coloco 0.3, vai diminuir a lr em 70%

check_best = True
check = True
early_stop = True
log = True
reduce_lr = True

callbacks_treino = callbacks(
    dir_modelos_salvos = dir_modelos_salvos, 
    dir_csv_log = dir_csv_log_treino,
    check_best = check_best, 
    check = check, 
    early_stop = early_stop, 
    log = log, 
    reduce_lr = reduce_lr,
    early_stop_epocas = early_stop_epocas,
    check_epocas = check_epocas,
    reduce_lr_epocas = reduce_lr_epocas,
    fator_reduce_lr = fator_reduce_lr
    )

callbacks adicionados:
check best
check 10
early stop
log
reduzindo lr


## 
<font size=5 color='white'> 4 - Treinar modelo

In [8]:
# ja foi comentado nos arquivos auxiliares
steps_per_epoch = len(dados_treino) // batch_size
validation_steps=len(dados_validacao) // batch_size

start_time = time.time()

model.fit(
    dataset_treino, 
    validation_data = dataset_validacao, 
    steps_per_epoch = steps_per_epoch, 
    validation_steps = validation_steps, 
    epochs = epocas,
    callbacks=callbacks_treino
    )

end_time = time.time()

tempo_treino = end_time - start_time
print(tempo_treino)

Epoch 1/100
339/339 [==============================] - ETA: 0s - loss: 1.5002 - categorical_accuracy: 0.3788 - precision: 0.6452 - recall: 0.0354 - auc: 0.6658
Epoch 1: val_loss improved from inf to 1.39554, saving model to ./redes/EfficientNetV2L/modelos_salvos\EfficientNetV2L-10_epoca_01.hdf5
339/339 [==============================] - 113s 255ms/step - loss: 1.5002 - categorical_accuracy: 0.3788 - precision: 0.6452 - recall: 0.0354 - auc: 0.6658 - val_loss: 1.3955 - val_categorical_accuracy: 0.4500 - val_precision: 0.7419 - val_recall: 0.0767 - val_auc: 0.7625 - lr: 1.0000e-05
Epoch 2/100
339/339 [==============================] - ETA: 0s - loss: 1.3402 - categorical_accuracy: 0.4696 - precision: 0.6648 - recall: 0.1418 - auc: 0.7729
Epoch 2: val_loss improved from 1.39554 to 1.26714, saving model to ./redes/EfficientNetV2L/modelos_salvos\EfficientNetV2L-10_epoca_02.hdf5
339/339 [==============================] - 90s 264ms/step - loss: 1.3402 - categorical_accuracy: 0.4696 - precisio

## 
<font size=5 color='white'> 5 - Avaliar modelo

In [ ]:
# ja foi comentado nos arquivos auxiliares
avaliar_modelo(
    model=model,
    rede=rede,
    id_rodada=id_rodada,  
    dataset_teste=dataset_teste,
    dados_teste=dados_teste,
    batch_size=batch_size,
    imagenet=imagenet,
    data_aug=data_aug,
    nome_otimizador=nome_otimizador,
    nome_loss=nome_loss,
    lr=lr,
    attention=attention,
    camada_pooling=camada_pooling,
    flatten=flatten,
    denses=denses,
    dropouts=dropouts,
    tempo_treino=tempo_treino
)


In [ ]:
dir_previsoes = './log_teste/previsoes'
for nome_arquivo in os.listdir(dir_previsoes):
    if nome_arquivo.startswith(f'resultados_{rede}-{id_rodada}'):
        nome_modelo = '_'.join(nome_arquivo.split('_')[1:]).replace('.csv', '')
        matriz_confusao(nome_modelo)